In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
cwd = os.getcwd()

fpath = "raw_datasets/erikson/Hindcast_1981_2/BTI_WavesAndStormSurges_1981-2100.csv"

with open(os.path.join(cwd, fpath)) as f:
    df = pd.read_csv(f)
    
df.head()

,start_date_of_storm_month,start_date_of_storm_day,start_date_of_storm_year,end_date_of_storm_month,end_date_of_storm_day,end_date_of_storm_year,Hso(m),Hs(m),Dp(deg),Tp(s),SS(m),Storm_duration(days),Hindcast_or_projection
0,8,29,1981,8,31,1981,2.3,2.1,235,7.4,0.26,2.4,hindcast
1,9,16,1985,9,20,1985,2.8,1.3,215,8.3,1.06,4.3,hindcast
2,9,17,1985,9,27,1985,3.2,3.5,305,9.5,1.06,9.6,hindcast
3,9,21,1986,9,23,1986,2.6,1.3,215,8.3,0.44,2.1,hindcast
4,8,24,1987,8,27,1987,2.2,2.5,315,7.4,0.52,3.3,hindcast


In [3]:
new_df = pd.DataFrame(
    df[["start_date_of_storm_year", "start_date_of_storm_month", "start_date_of_storm_day"]].values,
    columns=["Year", "Month", "Day"]
)

In [4]:
new_df["start"] = pd.to_datetime(new_df[["Year", "Month", "Day"]]).dt.normalize()

new_df['end'] = new_df['start'] + pd.to_timedelta(df['Storm_duration(days)'], unit='D').dt.round('H')

new_df = new_df.drop(columns=['Year', 'Month', 'Day'])

C:\Users\bruij_kn\AppData\Local\Temp\ipykernel_60564\1658008047.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_df['end'] = new_df['start'] + pd.to_timedelta(df['Storm_duration(days)'], unit='D').dt.round('H')


In [5]:
for col_name in df.columns:
    
    if not 'end' in col_name and not 'start' in col_name:
        
        new_df[col_name] = df[col_name]

Now that we have start and end times for all storms, let's transfer them to hour time stamps. If two storms overlap, we take the larger storm value.

In [6]:
df = pd.DataFrame(columns=new_df.columns.values)
df['time'] = pd.Series()

for i, row in new_df.iterrows():
        
    daterange = pd.date_range(row['start'], row['end'], freq='h')
    
    for date in daterange:
        
        row['time'] = date
        
        if not date in df.time.values:
            df = df._append(row)
            
        else:
            
            if row['Hso(m)'] > df[df.time==date]['Hso(m)'].values:
                
                df[df.time==date] = row
display(df)

C:\Users\bruij_kn\AppData\Local\Temp\ipykernel_60564\1967912532.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(row)


,start,end,Hso(m),Hs(m),Dp(deg),Tp(s),SS(m),Storm_duration(days),Hindcast_or_projection,time
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-29 00:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-29 01:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-29 02:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-29 03:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-29 04:00:00
...,...,...,...,...,...,...,...,...,...,...
518,2100-08-11,2100-08-13 12:00:00,4.1,1.8,255,8.3,0.00,2.5,projection,2100-08-13 08:00:00
518,2100-08-11,2100-08-13 12:00:00,4.1,1.8,255,8.3,0.00,2.5,projection,2100-08-13 09:00:00
518,2100-08-11,2100-08-13 12:00:00,4.1,1.8,255,8.3,0.00,2.5,projection,2100-08-13 10:00:00
518,2100-08-11,2100-08-13 12:00:00,4.1,1.8,255,8.3,0.00,2.5,projection,2100-08-13 11:00:00


In [7]:
display(df.iloc[50:60])

,start,end,Hso(m),Hs(m),Dp(deg),Tp(s),SS(m),Storm_duration(days),Hindcast_or_projection,time
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 02:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 03:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 04:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 05:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 06:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 07:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 08:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 09:00:00
0,1981-08-29,1981-08-31 10:00:00,2.3,2.1,235,7.4,0.26,2.4,hindcast,1981-08-31 10:00:00
1,1985-09-16,1985-09-20 07:00:00,2.8,1.3,215,8.3,1.06,4.3,hindcast,1985-09-16 00:00:00


In [9]:
save_path = os.path.join("ts_datasets/", "storms.csv")

df.to_csv(save_path)